# CSS 120: Environmental Data Science

## Climate Reanalysis

### Umberto Mignozzetti (UCSD)

(Based on Project Pythia and ClimateMatch)

# Today's Lecture

In the previous lectures, we examined atmospheric surface temperatures. 

Spatial variations in surface temperature created by uneven solar radiation (e.g., between the Equator and poles), are one of the main drivers of global-scale air movement. 

Other processes such as the Earth's rotation, storm tracks, and surface topography can also influence global wind patterns. 

By the end of this lecture, you will be able to:

- Describe the seasonal variations in surface winds.
- Calculate seasonal climatologies and create global maps.
- Compare statistics derived from these climatologies.

We will load the monthly-mean surface wind fields from ERA5 over a 30-year period.

## Packages

In [ ]:
# Just in case...

# !pip install pythia_datasets
# !pip install cartopy
# !pip install geoviews

# !pip install boto3 --quiet
# !pip install intake

## Packages

In [ ]:
from intake import open_catalog
import matplotlib.pyplot as plt
import matplotlib
import os
import pooch
import tempfile
import boto3
import botocore
import numpy as np
import xarray as xr
import warnings
import datetime
from cartopy import crs as ccrs, feature as cfeature
import ipywidgets as widgets

#  Suppress warnings issued by Cartopy when downloading data files
warnings.filterwarnings("ignore")

# Some display customizations
%config InlineBackend.figure_format = 'retina'
plt.style.use(
    "https://raw.githubusercontent.com/ClimateMatchAcademy/course-content/main/cma.mplstyle"
)

## Some auxiliary functions

In [ ]:
# Plotting Functions
def set_projection_figure(projection=ccrs.PlateCarree(), figsize=(5, 4.5)):
    # source:https://foundations.projectpythia.org/core/cartopy/cartopy.html

    projLccNY = projection  # ccrs.LambertConformal(central_longitude=cLon, central_latitude=cLat)

    fig = plt.figure(figsize=figsize)
    ax = plt.subplot(1, 1, 1, projection=projLccNY)

    format_axes(ax)
    # ax.add_feature(cfeature.STATES)
    # ax.add_feature(cfeature.RIVERS)
    return fig, ax

## Some auxiliary functions

In [ ]:
# Plotting Functions
def geographic_lon_to_360(lon):
    return 360 + lon

def inverted_geographic_lon_to_360(lon):
    return lon - 180
## Some auxiliary functions
def cbar_label(DD):
    return DD.attrs["long_name"] + " [" + DD.attrs["units"] + "]"

## Some auxiliary functions

In [ ]:
# Plotting Functions## Some auxiliary functions
def format_axes(ax):
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.LAKES, edgecolor="black", facecolor="None", alpha=0.3)
    gl = ax.gridlines(
        draw_labels=True, linewidth=1, color="black", alpha=0.5, linestyle="--"
    )
    gl.xlocator = matplotlib.ticker.MaxNLocator(7)
    gl.ylocator = matplotlib.ticker.MaxNLocator(5)
    gl.xlabels_top = False
    gl.ylabels_left = False
    # gl.xlines = False

## Some auxiliary functions

In [ ]:
# Helper functions
def pooch_load(filelocation = None, filename = None, processor = None):
    shared_location = "~/"
    user_temp_cache = tempfile.gettempdir()

    if os.path.exists(os.path.join(shared_location, filename)):
        file = os.path.join(shared_location, filename)
    else:
        file = pooch.retrieve(
            filelocation,
            known_hash=None,
            fname=os.path.join(user_temp_cache, filename),
            processor=processor,
        )

    return file

# Surface Winds

The large-scale atmospheric circulation significantly influences the climate we experience at different latitudes. 

The global atmospheric circulation is often depicted as a three-cell structure, as visualized [in this figure showing the surface winds](https://commons.wikimedia.org/wiki/File:Earth_Global_Circulation_-_en.svg):


This schematic of atmospheric circulation cells reveal meridional (north-south) and zonal (east-west) components of the large-scale surface winds. 

Let's see if you are able to detect these large-scale patterns in reanalysis data!
For this you will load ERA5 wind data from [Pangeo catalog](https://pangeo.io/catalog.html)

# Surface Winds

![Earth Global Circulation - en.svg](https://upload.wikimedia.org/wikipedia/commons/thumb/9/9c/Earth_Global_Circulation_-_en.svg/740px-Earth_Global_Circulation_-_en.svg.png?20190829153037).

## Annual Mean Wind Speed

You should start investigating this data by examining the global surface winds.

These winds are represented as vectors, consisting of a zonal component denoted as `u10` and a meridional component denoted as `v10`. 

The magnitude of the wind vector is given by:

\begin{align}
||u|| = \sqrt{u^2 + v^2}
\end{align}

To examine long-term changes in the wind field you should visualize the zonal wind component $u$ and the meridional wind component $v$ with monthly-mean data.

## Annual Mean Wind Speed

With the help of `xarray`, you can derive monthly means from higher-resolution data using the `xr.resample('1M').mean('time')` function.

For your convenience, we have already performed this step, and you can load the data using the following instructions:

- The variable `si10` represents the wind speed in this dataset.
- To calculate the long-term mean, we selected 30 years of data ranging from 1980 to 2010.

Let's grab the reanalysis data from before that we have prepocessed (noting naming convention changes):

## Annual Mean Wind Speed

In [ ]:
# note this can take a few minutes to download
filename_era5_mm = "ERA5_surface_winds_mm.nc"
url_era5_mm = "https://osf.io/3q4vs/download"

ERA5_mm = xr.open_dataset(pooch_load(url_era5_mm, filename_era5_mm)).sel(
    time=slice("2000", "2010")
)  # select the timeslice during loading
ERA5_mm

## Annual Mean Wind Speed

Now take the annual mean zonal surface wind speed.

To visualize this data, we will be using some *helper functions* defined in the beginning of the lecture just to set the map features and aesthetics.

Please take a look at them if you are interested in seeing how they work.

In [ ]:
# take the time mean
ERA5_ANN = ERA5_mm.mean("time")

## Annual Mean Wind Speed

In [ ]:
colorlevels_clim = np.arange(-10, 11, 1)

var = "u10"

fig, ax = set_projection_figure(
    projection=ccrs.PlateCarree(), figsize=(9, 5.5)
)

ax.set_title("Mean " + var, loc="left")

dataplot = ax.contourf(
    ERA5_ANN.longitude,
    ERA5_ANN.latitude,
    ERA5_ANN[var],
    levels=colorlevels_clim,
    transform=ccrs.PlateCarree(),
    cmap=plt.cm.coolwarm,
)

fig.colorbar(dataplot, orientation="vertical", label="m/s", shrink=0.55, pad=0.08)

## Annual Mean Wind Speed

In the zonal wind speed figure you created, there are two distinct wind bands between 35 to 65 degrees both north and south of the equator that blow from west to east (red, positive wind speeds). 

These mid-latitude wind bands are known as the *westerlies*. 

Additionally, you can see that winds predominantly blow from the east to the west (blue, negative wind speeds) in the tropics (less than 30° N/S), and are referred to as the *easterlies*.

## Annual Mean Wind Speed

### Your turn

Reproduce the previous figure, but modify it to plot **meridional wind** rather than **zonal wind**

In [ ]:
var = ...
fig, ax = set_projection_figure(projection=ccrs.PlateCarree(), figsize=(9, 5.5))
ax.set_title("Mean " + str(var), loc="left")
dataplot = ax.contourf(
    ERA5_ANN.longitude,
    ERA5_ANN.latitude,
    ERA5_ANN[var],
    levels=colorlevels_clim,
    transform=ccrs.PlateCarree(),
    cmap=plt.cm.coolwarm,
)
fig.colorbar(dataplot, orientation="vertical", label="m/s", shrink=0.55, pad=0.08)

## Annual Mean Wind Speed

There are strong southward winds in the subtropics of the Northern Hemisphere (blue, negative wind speed), and northward winds in the subtropics of the Southern Hemisphere (red, positive wind speed). 

The meridional winds are strongest on the western side of the continents. 

## Zonal-mean Wind Speed

To examine the latitudinal variation in the surface winds, you can plot the zonal mean of the annual mean zonal and meridional winds:

In [ ]:
fig, ax = plt.subplots(figsize=(9, 5.5))

ERA5_ANN.mean("longitude")["u10"].plot(
    label="zonal wind", ax=ax
)  # find the u10 mean value over the latitude

ERA5_ANN.mean("longitude")["v10"].plot(
    label="meridional wind", ax=ax
)  # find the v10 mean value over the latitude

ax.legend()  # add legend

ax.set_title("Annual mean, zonal mean winds", loc="left")  # add title
ax.set_ylabel("wind speed (m/s)")
ax.axhline(0, color="black")  # add a black line at x=0

## Monthly Climatology

Now, you should examine how the zonal mean winds vary throughout the seasons. 

Let us focus on the zonal wind component and create a special type of diagram called a Hoffmöller diagram.

In this diagram, the horizontal axis represents latitude, while the vertical axis represents time.

By using the Hoffmöller diagram, you can visualize how the average east-west winds change with each season.

## Monthly Climatology

In [ ]:
ERA5_mm.mean("longitude")["u10"].groupby("time.month").mean().plot()

## Monthly Climatology

### Your turn

Extend the above analysis to create a Hoffmöller diagram of the meridional winds.

In [ ]:
ERA5_mm.mean("longitude")[...].groupby("time.month").mean().plot()

## Monthly Climatology

- The winds in the Southern Hemisphere appear to be generally stronger compared to the Northern Hemisphere.

- The period between June and September shows strong meridional winds. 
    + These winds result from the seasonal variation of the Hadley cell. 
    + During the winter hemisphere in each respective hemisphere, the Hadley cell becomes much stronger, leading to the intensification of meridional winds.

## Monthly Climatology

Now let us analyze the seasons:

- We will plot the global map for DJF and JJA of the zonal wind. What do you see when you compare the mid-latitudes? (you can also plot their difference!)

- And then plot trend of the zonal mean zonal wind in DJF

More about this you can find in [Global Physical Climatology](https://www.atmos.washington.edu/~dennis/gpc.html), 
[The Atmospheric General Circulation](https://www.cambridge.org/us/academic/subjects/earth-and-environmental-science/atmospheric-science-and-meteorology/atmospheric-general-circulation?format=HB&isbn=9781108474245)
or the first few chapters of this evolving draft [Physics of Earth's Climate](https://climate-dynamics.org/wp-content/uploads/2017/04/Climate_Book.pdf)

In [ ]:
ERA5_season = ERA5_mm.groupby('time.season').mean()

var='u10'
season='DJF'
F, ax = set_projection_figure(projection = ccrs.PlateCarree(), figsize = (9, 5.5) )
ax.set_title('Zonal Wind DJF mean | '+ var , loc ='left')
dataplot = ax.contourf(ERA5_season.longitude, ERA5_season.latitude, ERA5_season.sel(season=season)[var],
                       levels = colorlevels_clim,
                       transform=ccrs.PlateCarree(), cmap= plt.cm.coolwarm)
_ = plt.colorbar(dataplot, orientation='vertical', label = 'm/s', shrink= 0.55 , pad = 0.08) # colorbar
plt.show()

## Monthly Climatology

In [ ]:
var='u10'
season='JJA'
F, ax = set_projection_figure(projection = ccrs.PlateCarree(), figsize = (9, 5.5) )
ax.set_title('Zonal Wind DJF mean | '+ var , loc ='left')
dataplot = ax.contourf(ERA5_season.longitude, ERA5_season.latitude, ERA5_season.sel(season=season)[var],
                       levels = colorlevels_clim,
                       transform=ccrs.PlateCarree(), cmap= plt.cm.coolwarm)
_ = plt.colorbar(dataplot, orientation='vertical', label = 'm/s', shrink= 0.55 , pad = 0.08) # colorbar
plt.show()

## Monthly Climatology

In [ ]:
var='u10'
pdata = ERA5_season.sel(season='DJF')[var] - ERA5_season.sel(season='JJA')[var]

F, ax = set_projection_figure(projection = ccrs.PlateCarree(), figsize = (9, 5.5) )
ax.set_title('Zonal Wind DJF mean - JJA mean | '+ var , loc ='left')
dataplot = ax.contourf(ERA5_season.longitude, ERA5_season.latitude,pdata ,
                       levels = colorlevels_clim,
                       transform=ccrs.PlateCarree(), cmap= plt.cm.coolwarm)
_ = plt.colorbar(dataplot, orientation='vertical', label = 'm/s', shrink= 0.55 , pad = 0.08) # colorbar
plt.show()

## Extra-tropical Storms

In the wind speed figure, you can notice areas of strong winds over the Southern Ocean, North Pacific, and North Atlantic.

These powerful winds are caused by weather systems known as extratropical storms or mid-latitude cyclones.

These storms occur in the middle latitudes, between 30 and 60 degrees north or south of the equator.

During winter, they are particularly strong over the Southern Ocean and the oceans in the Northern Hemisphere.

## Extra-tropical Storms

Extratropical storms form when warm and cold air masses interact. 

They have a low-pressure center and produce winds that circulate counterclockwise in the Northern Hemisphere and clockwise in the Southern Hemisphere. 

These storms can be intense, bringing strong winds, heavy rain, snow, and sleet. 

They often lead to problems like flooding, power outages, and disruptions in transportation.

## Extra-tropical Storms

The strength of these storms depends on factors such as the temperature difference between air masses, the speed of the jet stream, and the amount of moisture in the air. 

If you want to learn more about extratropical storms, you can refer to basic meteorology and atmospheric dynamics resources, or you can explore online sources such as the following:

- Wikipedia: [Extratropical Cyclone](https://en.wikipedia.org/wiki/Extratropical_cyclone)
- Pressbooks: [Chapter 13 - Extratropical Cyclones](http://pressbooks-dev.oer.hawaii.edu/atmo/chapter/chapter-13-extratropical-cyclones/)

Although an individual storm may last only a few days and doesn't create significant ocean currents, the continuous winds and the occurrence of multiple storms over a year can influence the development of ocean currents. 

These currents are a response to the persistent wind patterns.

## Questions?

## See you next class!